In [2]:
## Assignment 1 Intelligent Systems 

In [3]:
## Matrixes 

- preferences Matrix
- frienship Matrix
- authorship Matrix 
OUT: 
Assignment Matrix : 
Rows Reviewers 
Cols Papers 

SyntaxError: invalid syntax (2459903027.py, line 3)

In [12]:
import pygad
import pandas as pd
import numpy as np

# Load the JSON file into a pandas DataFrame


In [24]:
#  Converting the preference data into an appropriate format 


df = pd.read_json('easy_dataset_1.json')
#First drop unnecessary data and the them into variables 
num_papers = df.loc[0,"num_papers"]
df = df.drop(columns="num_papers")
num_reviewers = df.loc[0,"num_reviewers"]
df = df.drop(columns="num_reviewers")
reviewer_capacity = df.loc[0,"reviewer_capacity"]
df = df.drop(columns ="reviewer_capacity")
min_reviews_per_paper = df.loc[0,"min_reviews_per_paper"]
df = df.drop(columns="min_reviews_per_paper")
max_reviews_per_paper = df.loc[0,"max_reviews_per_paper"]
df = df.drop(columns="max_reviews_per_paper")
df = df.drop(columns="dataset_id")
#df.rename(columns={"dataset_id" : "reviewer"}, inplace=True)
#df.rename(index={0: "reviewer1", 1: "reviewer2", 2: "reviewer3", 3: "reviewer4", 4: "reviewer5"})
print(df)


#try also with numpy arrays 

preferences_matrix = np.array(df["preferences"].tolist())
friendships_matrix = np.array(df["friendships"].tolist())
authorship_matrix = np.array(df["authorship"].tolist())
print(authorship_matrix)
print("reviewercap:", reviewer_capacity)


       preferences      friendships       authorship
0  [3, 2, 1, 5, 3]  [0, 0, 0, 0, 0]  [0, 0, 0, 0, 0]
1  [5, 2, 4, 2, 2]  [0, 0, 1, 0, 0]  [0, 1, 0, 0, 0]
2  [4, 3, 1, 2, 4]  [0, 1, 0, 0, 0]  [0, 0, 0, 0, 0]
3  [4, 2, 2, 1, 1]  [0, 0, 0, 0, 0]  [1, 1, 0, 0, 1]
4  [4, 3, 4, 1, 5]  [0, 0, 0, 0, 0]  [0, 0, 1, 1, 0]
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [1 1 0 0 1]
 [0 0 1 1 0]]
reviewercap: 3


In [26]:
# Deciding on an effective solution representation
# Assignments Reprensation 

#reviewer x papers 5 x 5 
#not sure if numpy is better here but we will work with this 

assign_matrix = pd.DataFrame(0, index=[f"Reviewer {i+1}" for i in range(5)], 
                                columns=[f"Paper {j+1}" for j in range(5)])

assign_matrixnp = np.zeros((5,5), dtype= int)
assign_matrixnp[0,2] = 1
assign_matrixnp[0,3] = 1
assign_matrixnp[0,4] = 1
assign_matrixnp[0,1] = 1
assign_matrixnp[1,2] = 1
assign_matrixnp[2,4] = 1
assign_matrixnp[3,0] = 1
assign_matrixnp[4,3] = 1
print(assign_matrixnp)

[[0 1 1 1 1]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 1 0]]


In [37]:
#compute fitness value in absolute or relative 
def f(matrix): 
    fit_val = 0 

    
    # penalty scores table/distribution
    preference_points = np.array([[0,1,2,3,4,5],[3,2,1,0,-1,-2]])
    penalty_reviewer_cap = 3
    penalty_authorship = 7
    penalty_friendship = 5

    #iterate over the rows
    for i in range(len(assign_matrixnp[:,0])): 
        #iteratre over the columns 
        for j in range(len(assign_matrixnp[0,:])): 
            #print(fit_val)
            if assign_matrixnp[i,j] == 1: 
                 index = preferences_matrix[i,j]
                 fit_val += preference_points[1, index]
    
    # penalties for reviewer constraints violations              
    for i in range(len(assign_matrixnp[:,0])): 
        x = np.sum(assign_matrixnp[i,:])
        if x > reviewer_capacity:
            fit_val += penalty_reviewer_cap
            print("penalty")

    # not very efficient like this integrate in other for loop later 
    # penalties for friendship constraints violation 
    # iterate over friendships or iterate over assignments -> assignments seems more intuitiv
    for i in range(len(assign_matrixnp[:,0])): 
        for j in range(len(assign_matrixnp[0,:])): 
            #authorship penalty -> if reviewer assign is also the author give penalty
            if assign_matrixnp[i,j] == 1 and authorship_matrix[i,j] ==1:
                fit_val+= penalty_authorship
            #friendship penalty 
            if assign_matrix[i,j] == 1: 
                reviewer = i 
                paper = j 
                for y in range(len(assign_matrixnp[0,:])): 
                    if authorship_matrix[y,j] == 1:
                        author = y 
                    if friendship_matrix[y,i] == 1: 
                        fit_val+= penalty_friendship
    return fit_val 

print(f(assign_matrixnp))

def fitness_function(ga_instance, solution, solution_idx):
    return f(solution)[0]   # solution is a numpy array with length 1. We return it as a scalar value



penalty


KeyError: (0, 0)